In [ ]:
import os
import numpy as np
import copy
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# switch working dir to project root (found via Git)
import os
import subprocess

root_dir = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip()
print(root_dir)
os.chdir(root_dir)

In [ ]:
# Stolen from https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/vis.py
def vis_seg(img, seg, palette, alpha=0.5):
    """
    Visualize segmentation as an overlay on the image.
    Takes:
        img: H x W x 3 image in [0, 255]
        seg: H x W segmentation image of class IDs
        palette: K x 3 colormap for all classes
        alpha: opacity of the segmentation in [0, 1]
    Gives:
        H x W x 3 image with overlaid segmentation
    """
    vis = np.array(img, dtype=np.float32)
    mask = seg > 0
    vis[mask] *= 1. - alpha
    vis[mask] += alpha * palette[seg[mask].flat]
    vis = vis.astype(np.uint8)
    return vis

Choose experiment settings to view results.

In [ ]:
from revolver.data.davis import DAVISInstSeg

# Exp Config
exp = 'cofeat-late-davis-diossbdft-fixed-lr5'
it = '100000'
model = 'cofeat-late'
dataset = 'davis'
datatype = 'late-conditional-video'
split = 'val'
count = -1
shot = 2
seed = 1337

data_root = f"data/{dataset}"
exp_dir = f"./experiments/{exp}/"
count_ = 'dense' if count == -1 else "{}sparse".format(count) if count else 'randsparse'
output_dir = '{}{}-iter{}-{}-{}-{}-{}-{}shot-{}'.format(exp_dir, model, it, dataset, datatype, split, count_, shot, seed)
print('Loading segmentations from:\n', output_dir)

ds = DAVISInstSeg()

seqs = open(f"{data_root}/ImageSets/2017/val.txt", 'r').read().splitlines()
print(f'Sequences to choose from: {seqs}')

Choose sequence to view segmentations.

In [ ]:
vid = 'bike-packing'
frms = sorted(os.listdir(f"{output_dir}/{vid}"))

start = 0
num_show = 6

Show image, ground truth seg, and prediction for subset `num_show` of sequence frames. Walk through the sequence in order, looping back to the beginning once the end is reached, overlaying the ground truth and predicted segmentations on the image.

In [ ]:
end = start + num_show
k = min(len(frms) - start, num_show)

fig, axes = plt.subplots(k,3, figsize=(30, 30))
fd = {'fontsize': 30}
axes[0,0].set_title('Original Video Frame', fontdict=fd)
axes[0,1].set_title('Ground Truth Segmentation', fontdict=fd)
axes[0,2].set_title('Predicted Segmentation', fontdict=fd)

print(output_dir)
print('Video:', vid)
print(f"Frames {start} through {start +k}")

for i, frm in enumerate(frms[start: start + k]):
    # load image, gt seg, and predicted seg
    img = Image.fromarray(ds.load_image(ds.slug_to_image_path((vid, frm[:-4]))))
    gt = ds.load_annotation(ds.slug_to_annotation_path((vid, frm[:-4])))
    seg = np.array(Image.open(f"{output_dir}/{vid}/{frm}"), dtype=np.uint8)
    
    # overlay segs on image
    seg = vis_seg(img, seg, ds.palette, alpha=0.5)
    gt = vis_seg(img, gt, ds.palette, alpha=0.5)
    
    axes[i, 0].imshow(img)
    axes[i, 1].imshow(gt)
    axes[i, 2].imshow(seg)
    
    for _, ax in np.ndenumerate(axes):
        ax.set_axis_off()
    
plt.tight_layout()

start += num_show
if end >= len(frms):
    start = 0

Palette and overlay segmentations for all the sequences in the dataset and resave.

In [ ]:
counter = 0

seg_dir = f"{output_dir}-overlay"
os.makedirs(seg_dir, exist_ok=True)

for vid in videos:
    frms = sorted(os.listdir(f"{output_dir}/{vid}"))
    for i, frm in enumerate(frms):
        # skip existing
        if os.path.isfile(f"{seg_dir}/{frm}"):
            continue
        img = Image.fromarray(ds.load_image(ds.slug_to_image_path((vid, frm[:-4]))))
        seg = np.array(Image.open(f"{output_dir}/{vid}/{frm}"), dtype=np.uint8)
        seg = Image.fromarray(vis_seg(img, seg, ds.palette, alpha=0.5))
        seg.save(f"{seg_dir}/{frm[:-4]}.jpg")